In [1]:
def clip(tmp1):
    import subprocess
    import shlex  # 导入 shlex 模块
    # 使用 shlex.quote 来转义 inp 字符串
    tmp2 = str(tmp1)
    safe_str = shlex.quote(tmp2)
    subprocess.run('echo {} | wclip'.format(safe_str), shell=True)  

def cvin(k):
    clip(In[k])
    
import numpy as np
import matplotlib.pyplot as plt

import time
#from tqdm import tqdm  # tqdm是显示循环进度条的库
from tqdm.notebook import tqdm #推荐在jupyter中使用自带的进度条

np.random.seed(0) #重置种子为0

import copy #复制方法
#格式化输出
np.set_printoptions(precision=3, suppress=True, linewidth=100)

### 基于模型的强化学习 & 无模型的强化学习
强化学习算法分为两种：
- 基于模型的强化学习（model-based reinforcement learning）
- 无模型的强化学习（model-free reinforcement learning）

无模型的强化学习根据智能体与环境交互采样到的数据直接进行策略提升或者价值估计，而有模型就是已知的，比如策略迭代和价值迭代

#### 强化学习评价指标
强化学习算法有两个重要的评价指标：
- 一个是算法收敛后的策略在初始状态下的 **期望回报**
- 另一个是**样本复杂度**，即算法达到收敛结果需要在真实环境中采样的样本数量

模型通常会更低样本复杂度，但是—— **_环境模型可能并不准确_**
> 因此基于模型的强化学习算法收敛后其策略的期望回报可能不如无模型的强化学习算法

### 1.Dyna-Q—>模拟数据和真实数据一起改进策略
Dyna-Q 使用一种叫做 Q-planning：基于模型生成一些模拟数据，然后用——
**_——模拟数据和真实数据一起改进策略_**
- 每次选取一个曾经访问过的状态s
- 曾经在该状态下执行过的动作a
- 通过模型得到转移后的状态$(s^{\prime},r)$

最终得到一个四元组$(s,a,r,s^{\prime})$，用 Q-learning 的更新方式来更新动作价值函数。
![image.png](attachment:image.png)

#### 算法流程
![image-2.png](attachment:image-2.png)
在每次与环境进行交互执行一次 Q-learning 之后，Dyna-Q 会做n次 Q-planning，其中 Q-planning 的次数N是一个事先可以选择的超参数，**当其为 0 时就是普通的 Q-learning**

> 本质来说，Q-learning就是真实环境，每次只能一步一步走，跳崖就结束了。而Dyna-Q则是一个可以被玩坏的环境，反复访问同一个状态$s_m$

### Dyna-Q算法实现：
最主要的修改是加入了环境模型`model`，用一个字典表示，每次在真实环境中收集到新的数据，就把它加入字典
根据字典的性质，若该数据本身存在于字典中，便不会再一次进行添加。在 Dyna-Q 的更新中，执行完 Q-learning 后，会立即执行 Q-planning。

核心算法实现：
```python
def q_learning(self, s0, a0, r, s1) #略
def update(self, s0, a0, r, s1):
    self.q_learning(s0, a0, r, s1)
    self.model[(s0, a0)] = r, s1  # 将数据添加到模型中
    for _ in range(self.n_planning):  # Q-planning循环
        # 随机选择曾经遇到过的状态动作对
        (s, a), (r, s_) = random.choice(list(self.model.items()))
        self.q_learning(s, a, r, s_)
```
> ?在Q-learning中，"放回池"巧妙利用贝尔曼最大期望实现了对历史数据的使用，那么Dyna-Q，"model"是否就是"放回池"本身？

> **_注意：Q-planning循环是随机选取的，与当前s,a毫无关系！！！！_**

#### Dyna-Q效果
差异：Q-planning循环次数
![image-3.png](attachment:image-3.png)

从上述结果中我们可以很容易地看出，随着 Q-planning 步数的增多，Dyna-Q 算法的收敛速度也随之变快。当然，并不是在所有的环境中，都是 Q-planning 步数越大则算法收敛越快，这取决于环境是否是确定性的，以及环境模型的精度。

> 在上述悬崖漫步环境中，**状态的转移是完全确定性的** ，构建的环境模型的精度是最高的，所以可以通过增加 Q-planning 步数来直接降低算法的样本复杂度

### 总结：
直观地展示了 Q-planning 步数对于收敛速度的影响。我们发现基于模型的强化学习算法 Dyna-Q 在以上环境中获得了很好的效果，但这些环境比较简单，模型可以直接通过经验数据得到。
如果环境比较复杂，状态是连续的，或者状态转移是随机的而不是决定性的，如何学习一个比较准确的模型就变成非常重大的挑战，这直接影响到基于模型的强化学习算法能否应用于这些环境并获得比无模型的强化学习更好的效果。

> 智能体和模拟环境，都是两个AI